In [11]:
from collections import defaultdict
from pprint import pprint
RRF_K = 60 # RRFの平滑化定数
def weighted_reciprocal_rank_fusion(
    rankings: list[list[str]],  # 複数のランキングIDリスト (例: [bm25_results, vector_results])
    weights:  list[float] = (0.5, 0.5)  # 各ランキングの重み (例: [0.6, 0.4])
) -> list[tuple[str, float]]:
    """
    二組のランキング情報とその重みを受け取り、RRFで統合スコアを計算する関数 (Weighted RRF)。
    Args: rankings: 各リトリーバーから得られた文書IDのランキングリストのリスト。
                     [['ID1','ID2','ID3'], ['ID2','ID3','ID1']]
          weights: 各ランキングに対応する重みのリスト。
                     [0.6, 0.4]
    Returns: 文書IDと最終スコアのタプルを要素とする、スコア降順のリスト。
    """

    fused_scores: Dict[str, float] = defaultdict(float)

    # 1. 各リトリーバーのランキングをループし、スコアを計算
    for i, rank_list in enumerate(rankings):
        weight = weights[i]
        # リスト内の各文書とその順位をループ
        for rank, doc_id in enumerate(rank_list):
            # 順位は0から始まるため、r_i は rank + 1
            r_i = rank + 1 
            # Weighted RRFスコアを計算し、合計する
            # S_d = w_i * [ 1 / (r_i + k) ]
            score_contribution = weight * (1 / (r_i + RRF_K))
            fused_scores[doc_id] += score_contribution

    # 2. スコアが高い順にソート
    # 結果をタプル (文書ID, スコア) のリストとして返す
    sorted_ranking = sorted(
        fused_scores.items(), key=lambda item: item[1], reverse=True
    )

    return sorted_ranking

pprint(weighted_reciprocal_rank_fusion([['ID1','ID2','ID3'], ['ID2','ID3','ID1']], [0.5, 0.5]))
pprint(weighted_reciprocal_rank_fusion([['ID1','ID2','ID3'], ['ID2','ID3','ID1']], [0.7, 0.3]))

[('ID2', 0.01626123744050767),
 ('ID1', 0.016133229247983348),
 ('ID3', 0.016001024065540194)]
[('ID1', 0.016237314597970336),
 ('ID2', 0.016208355367530406),
 ('ID3', 0.015949820788530463)]
